## FCS Lab 2 Submission Report

* Name of Student(s): Shoham Chakraborty
* Student ID(s): 1004351

In [2]:
import base64
import requests
def XOR(a, b):
    r = b""
    for x, y in zip(a, b):
        r += (x ^ y).to_bytes(1, "big")
    return r
class Client:
    def __init__(self, endpoint, uid):
        self.endpoint = endpoint
        self.uid = str(uid).lower().strip()
    def post(self, url, data=None):
        r = requests.post(url, json=data).json()
        if not r["success"]:
            print("Warning: something might be wrong with the server")
            print("If you don't think is your mistake, please report it!")
        return r
    def get_story_cipher(self):
        url = self.endpoint+"/story"
        return requests.get(url).json()
    def post_story_plaintext(self, solution):
        url = self.endpoint+"/story"
        solution = str(solution).lower().strip()
        data = {"solution":solution}
        return self.post(url, data)
    def get_score_msg_cipher(self):
        url = self.endpoint+"/score"
        data = {"request":"get_msg", "id":self.uid}
        return self.post(url, data)
    def submit_score_msg_cipher(self, cipher_base64):
        url = self.endpoint+"/score"
        data = {"request":"decrypt_msg", "id":self.uid, "cipher": cipher_base64}
        return self.post(url, data)
    def base64_encode_bytes(self, byte_array):
        return str(base64.b64encode(byte_array))[2:-1]
    def base64_decode_bytes(self, base64_string):
        return base64.b64decode(base64_string)

In [3]:
endpoint = "http://35.197.130.121/"
uid = "ayylmao123"

client = Client(endpoint, uid)

## Part I: Story - Substitution Cipher

1. GET the cipher for the story
2. Crack this with frequency analysis
3. POST it back to the server to check (example is provided below)

If the response contains `'solution_correct': 'correct'`, then your decryption is correct. Otherwise, a distance will be provided to let you know how far off you are. If you are off by a tiny bit (say, 1 or 2), you can check things like line-ending, extra space at start/end etc. The verification is not case sensitive.

In [4]:
story_cipher = client.get_story_cipher()["cipher"]
print("story_cipher:", story_cipher[:50], "...")

story_cipher: MXQJ YI IOCFXEWUQH. VEH Q BEEEEEDW, BEEEDW JYCU Y  ...


In [5]:
# we convert the cipher using a dictionary
conversion_dictionary = {
    "A": "k",
    "B": "l",
    "C": "m",
    "D": "n",
    "E": "o",
    "F": "p",
    "G": "q",
    "H": "r",
    "I": "s",
    "J": "t",
    "K": "u",
    "L": "v",
    "M": "w",
    "N": "x",
    "O": "y",
    "P": "z",
    "Q": "a",
    "R": "b",
    "S": "c",
    "T": "d",
    "U": "e",
    "V": "f",
    "W": "g",
    "X": "h",
    "Y": "i",
    "Z": "j",
}

decrypted = ""
for ch in story_cipher:
    try:
        decrypted += conversion_dictionary[ch]
    except KeyError:
        decrypted += ch # puncuation
        # print(f"Dictionary entry does not exist for character {ch}")


print(decrypted)

what is symphogear. for a looooong, looong time i have never bothered engaging myself in this franchise. i did not understand what it is. now that the show is having its last season, i decided to finally give in, give symphogear a try from the very start. i wondered how have i missed out on the anime of the decade all these years. i honestly did not know what to expect watching the very first episode knowing absolutely nothing about the franchise. the show toyed with my emotions so much in that opening setpiece. it ended up being one of the most life affirming shows out there. a stunning display of idiocy and action that is both charming and captivating. it is confident in its strengths and parades its weaknesses proudly, a show that is both all style and all substance. oh but most of all, it is a true roller coaster of emotions, and i do not use that term lightly. i laughed, i cried, i got frustrated at the ineptitude and stupidity of both the characters and the creators, but most of 

In [6]:
# example posting a string back to the server
client.post_story_plaintext(decrypted)
# a distance is provided for you to check how close you are

{'distance': '0', 'solution_correct': 'correct', 'success': True}

In [7]:
# you can also load solution from a text file
with open("./solution.txt", "r") as file:
    PART_1_SOLUTION = file.read()
part_1_result = client.post_story_plaintext(PART_1_SOLUTION)
print(part_1_result)
assert part_1_result["solution_correct"] == "correct"

{'distance': '0', 'solution_correct': 'correct', 'success': True}


## Part II: Changing the Score Message - OTP

In [8]:
response = client.get_score_msg_cipher()
print(response)

{'cipher': 'WzO3XfKl1Ln8zTzUVu3iRO2NCZeHnaWRZfcOybrT8e6DnfNJUoJdpNOUAuEnaZ8=', 'hint': 'it is a OTP, you will not be able to guess it, find a way to edit the message without the OTP key', 'success': True}


In [9]:
cipher:bytearray = client.base64_decode_bytes(response["cipher"])
print(cipher)

b"[3\xb7]\xf2\xa5\xd4\xb9\xfc\xcd<\xd4V\xed\xe2D\xed\x8d\t\x97\x87\x9d\xa5\x91e\xf7\x0e\xc9\xba\xd3\xf1\xee\x83\x9d\xf3IR\x82]\xa4\xd3\x94\x02\xe1'i\x9f"


In [10]:
encoded_cipher = client.base64_encode_bytes(cipher)
client.submit_score_msg_cipher(encoded_cipher)

{'plaintext': 'Student ID ayylmao123 gets a total of 0 points!',
 'success': True}

In [11]:
def hax():
    m = bytearray.fromhex(f"Student ID {uid} gets a total of 0 points!".encode().hex())
    m1 = bytearray.fromhex(f"Student ID {uid} gets a total of 9 points!".encode().hex())

    mask = XOR(m, m1)
    return XOR(cipher, mask)

In [12]:
new_cipher = hax()
encoded_new_cipher = client.base64_encode_bytes(new_cipher)
client.submit_score_msg_cipher(encoded_new_cipher)

{'plaintext': 'Student ID ayylmao123 gets a total of 9 points!',
 'success': True}